# Holistic Health AI

In [2]:
!pip install langchain openai faiss-cpu chromadb tiktoken python-dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ------- -------------------------------- 2.6/13.7 MB 13.7 MB/s eta 0:00:01
   ---------------- ----------------------- 5.

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "Write your Open AI API Key Here"

### Let's test if it works using the OpenAI API with GPT-3.5-Turbo

In [9]:
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are a holistic health assistant."},
              {"role": "user", "content": "Give me one quick holistic health tip."}],
    max_tokens=100 
)

print(response.choices[0].message.content)

Practice deep breathing exercises to reduce stress and promote relaxation. Deep belly breaths can help calm your mind, improve oxygen flow in your body, and support overall well-being. Try taking a few minutes each day to focus on your breath and notice the difference it can make in your stress levels.


## Creating a Vector Database with FAISS

In [26]:
import faiss
import numpy as np
import openai
from openai import OpenAI
from typing import List

#### Define Sample Holistic Health Texts

In [47]:
documents = [
    "Drinking warm lemon water in the morning helps detoxify your liver and improves digestion.",
    "Grounding, or walking barefoot on natural surfaces, can help reduce inflammation and improve sleep quality.",
    "Intermittent fasting can boost mitochondrial function and support cellular repair.",
    "Chamomile tea is known for its calming effects and can help with sleep and anxiety reduction.",
    "Exposing yourself to morning sunlight helps regulate your circadian rhythm and improves mood.",
    "Cold showers can boost circulation, improve immune function, and increase mental alertness.",
    "Eating fermented foods like kimchi and yogurt supports gut microbiome health.",
    "Practicing gratitude daily can reduce stress and improve overall mental health.",
    "Turmeric has powerful anti-inflammatory properties and supports joint health.",
    "Deep breathing exercises help lower cortisol levels and promote relaxation.",
    "Drinking green tea can enhance brain function and metabolic rate.",
    "A diet rich in omega-3 fatty acids supports heart and brain health.",
    "Magnesium deficiency can lead to muscle cramps and sleep disturbances.",
    "Spending time in nature can reduce anxiety and improve overall well-being.",
    "Getting enough sleep is crucial for hormone regulation and cognitive function.",
    "Probiotics support gut health by balancing beneficial bacteria.",
    "Blue light exposure at night disrupts melatonin production and sleep quality.",
    "Seed cycling can help regulate estrogen and progesterone balance in women.",
    "Sweating through exercise or sauna supports detoxification by eliminating heavy metals.",
    "Cold exposure can stimulate mitochondrial biogenesis, improving energy production.",
    "Journaling can improve mental clarity and emotional well-being.",
    "Avocados, MCT oil, and omega-3 and omega-9 fats promote satiety and help reduce belly fat.",
    "Berries are rich in antioxidants and fiber, helping to reduce oxidative stress and support gut health.",
    "Chia seeds expand in the stomach, promoting fullness due to their fiber and healthy fat content.",
    "Regular exercise reduces the risk of depression and anxiety while enhancing cognitive function.",
    "Quality sleep improves insulin sensitivity and regulates appetite-controlling hormones.",
    "Nuts and seeds are packed with protein, fiber, and essential vitamins, reducing heart disease risk.",
    "Exposure to natural light regulates circadian rhythm and enhances mood stability.",
    "Coffee is rich in antioxidants and may lower the risk of Parkinson’s and Alzheimer’s diseases.",
    "Fatty fish provide omega-3 fatty acids that support heart and brain health and reduce dementia risk.",
    "Avoiding ultra-processed foods lowers the risk of obesity and type 2 diabetes.",
    "Consuming probiotic-rich yogurt helps balance gut microbiota and improve digestion.",
    "Green tea improves cognitive function and boosts metabolism naturally.",
    "Daily meditation helps manage stress and improve concentration levels.",
    "Spending time in nature reduces anxiety and promotes overall well-being.",
    "Staying hydrated is essential for optimal body functions and energy levels."
]

documents.extend([
    "Holistic health approaches emphasize the integration of mind, body, and spirit to promote overall well-being.",
    "Incorporating spiritual care into nursing practices can enhance patient outcomes and satisfaction.",
    "Traditional healing practices often involve a holistic approach, addressing physical, emotional, and spiritual aspects of health.",
    "Holistic nursing care involves considering patients' psychological and social needs alongside their physical health.",
    "Complementary therapies, such as meditation and acupuncture, are integral components of holistic health practices."
])

### Converting Texts into Vectors Using the OpenAI Embedding Model

In [50]:
import openai
import numpy as np

openai.api_key = os.getenv("OPENAI_API_KEY")

def get_embeddings(texts):
    response = openai.embeddings.create(
        model="text-embedding-ada-002",
        input=texts
    )
    return np.array([item.embedding for item in response.data])


document_embeddings = get_embeddings(documents)

### Creating a FAISS Index and Adding Vectors

In [55]:
import faiss

if 'document_embeddings' not in globals():
    raise ValueError("Error: 'document_embeddings' variable is not defined. Please run the embedding generation step first.")

embedding_dim = document_embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(document_embeddings).astype('float32'))

print(f"FAISS index contains {index.ntotal} vectors.")

FAISS index contains 41 vectors.


### Processing Queries and Finding the Nearest Neighbors

In [57]:
def search(query, k=5):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    results = [documents[i] for i in indices[0]]
    return results

query = "How can I improve my sleep quality?"
results = search(query)
for result in results:
    print(result)

Quality sleep improves insulin sensitivity and regulates appetite-controlling hormones.
Grounding, or walking barefoot on natural surfaces, can help reduce inflammation and improve sleep quality.
Getting enough sleep is crucial for hormone regulation and cognitive function.
Blue light exposure at night disrupts melatonin production and sleep quality.
Exposing yourself to morning sunlight helps regulate your circadian rhythm and improves mood.


## Updating the Model for Answer Generation with GPT-3.5

In [74]:
def generate_combined_response(query, top_k=5):
    query_embedding = get_embeddings([query])
    _, indices = index.search(np.array(query_embedding).astype('float32'), top_k)
    relevant_docs = [documents[i] for i in indices[0]]
    prompt = (
        "You are a holistic health expert. Answer the user's question using both the provided insights and your own knowledge.\n\n"
        "Provided insights:\n"
        f"{' '.join(relevant_docs)}\n\n"
        "User query: {query}\n\n"
        "Response:"
    )
    
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a holistic health expert with access to both external insights and general knowledge."},
                  {"role": "user", "content": prompt}],
        max_tokens=150
    )

    return response.choices[0].message.content

query = "What are natural ways to improve sleep?"
response = generate_combined_response(query)
print(response)

It seems like you are looking for natural ways to improve your sleep quality and reduce inflammation. One effective method is grounding, which involves walking barefoot on natural surfaces. This practice can help reduce inflammation in the body and promote better sleep by connecting you with the Earth's energy. Improved sleep quality, in turn, can enhance your insulin sensitivity and regulate appetite-controlling hormones, contributing to overall health and well-being.

In addition to grounding, exposure to natural light is essential for regulating your circadian rhythm, which is crucial for maintaining a healthy sleep-wake cycle. Morning sunlight, in particular, can have a positive impact on your mood and overall well-being. Incorporating outdoor activities or simply spending time in natural light can help support your body's natural rhythms


## Model Optimization

#### Enhancing the Model to Provide More Scientific Responses

In [78]:
def generate_scientific_response(query, top_k=5):
    query_embedding = get_embeddings([query])
    _, indices = index.search(np.array(query_embedding).astype('float32'), top_k)
    relevant_docs = [documents[i] for i in indices[0]]
    prompt = (
        "You are an expert in holistic health and functional medicine. "
        "Please provide a scientific, well-structured response to the following query. "
        "Use formal language and, where possible, reference scientific mechanisms or processes involved.\n\n"
        f"Relevant insights from research:\n"
        f"{' '.join(relevant_docs)}\n\n"
        f"User query: {query}\n\n"
        "Provide a structured response, ensuring clarity and scientific rigor."
    )

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a holistic health expert with a deep understanding of functional medicine. Respond in a well-structured, scientific manner."},
                  {"role": "user", "content": prompt}],
        max_tokens=200  # Slightly increased for better explanations
    )

    return response.choices[0].message.content

query = "How does intermittent fasting improve mitochondrial function?"
response = generate_scientific_response(query)
print(response)

Intermittent fasting is a dietary strategy that involves cycling between periods of eating and fasting. Research has shown that intermittent fasting can improve mitochondrial function through several mechanisms:

1. **Mitochondrial Biogenesis**: Mitochondria are the powerhouses of the cell responsible for producing energy in the form of adenosine triphosphate (ATP). Intermittent fasting has been shown to stimulate mitochondrial biogenesis, which is the process of creating new mitochondria. This increase in mitochondrial mass enhances the cellular capacity for energy production.

2. **Autophagy**: Intermittent fasting triggers a process known as autophagy, which is the body's way of cleaning out damaged cells and regenerating new ones. Autophagy plays a crucial role in mitochondrial quality control by eliminating dysfunctional mitochondria and promoting the synthesis of new healthy mitochondria.

3. **Metabolic Switch**: During fasting periods, when glucose levels are low, the body swit

#### Expanding Dataset for More Comprehensive Coverage

In [86]:
documents.extend([
    "Intermittent fasting enhances autophagy, a process that clears damaged cells and improves mitochondrial efficiency.",
    "Cold exposure stimulates mitochondrial biogenesis, enhancing energy production and metabolic efficiency.",
    "Chronic inflammation is linked to mitochondrial dysfunction, leading to oxidative stress and cellular aging.",
    "Circadian rhythm misalignment can increase the risk of metabolic disorders, including insulin resistance and obesity.",
    "A balanced gut microbiome is crucial for neurotransmitter production, impacting mood and cognitive function.",
    "Omega-3 fatty acids play a vital role in reducing neuroinflammation and supporting brain plasticity.",
    "Vitamin D is essential for immune regulation and has been linked to improved mitochondrial function.",
    "Exposure to early morning sunlight helps synchronize the circadian clock, improving sleep quality and hormonal balance.",
    "Ketogenic diets have been shown to enhance mitochondrial function by increasing beta-oxidation of fatty acids.",
    "Polyphenols found in berries and green tea have antioxidant properties that protect mitochondria from oxidative damage.",
    "Infrared sauna therapy can promote detoxification and improve mitochondrial energy production.",
    "Regular exercise increases mitochondrial density, improving endurance and overall metabolic health.",
    "Melatonin is not only a sleep regulator but also a potent antioxidant that protects mitochondrial DNA from oxidative stress.",
    "Excessive blue light exposure at night suppresses melatonin production, disrupting sleep and circadian regulation.",
    "Probiotic-rich foods like kefir and sauerkraut help maintain gut microbiome diversity, supporting digestion and immune function."
])

#### FAISS Optimization (Smarter Search)

In [99]:
def generate_optimized_response(query, top_k=5, score_threshold=0.3):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), top_k)
    relevant_docs = [
        documents[i] for i, d in zip(indices[0], distances[0]) if d < score_threshold
    ]

    if not relevant_docs:
        relevant_docs = ["No relevant documents found in our database. Please use general knowledge."]

    prompt = (
        "Use the following holistic health insights to answer the user's question. "
        "If no relevant information is available, rely on your own expertise:\n\n"
        f"{' '.join(relevant_docs)}\n\n"
        f"User query: {query}\n\n"
        "Provide a structured and scientifically accurate response."
    )

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a holistic health expert. Provide responses in a scientific, well-structured manner."},
                  {"role": "user", "content": prompt}],
        max_tokens=200
    )

    return response.choices[0].message.content

query = "How do omega-3 fatty acids support brain health?"
response = generate_optimized_response(query)
print(response)

Omega-3 fatty acids are essential nutrients that play a crucial role in brain health. They are a type of polyunsaturated fat found in high amounts in fatty fish such as salmon, mackerel, and sardines. Here is how Omega-3 fatty acids support brain health:

1. **Structural Component of Brain Cells**: Omega-3 fatty acids are a key component of cell membranes in the brain. They help maintain the fluidity and flexibility of cell membranes, which is essential for proper communication between brain cells.

2. **Anti-Inflammatory Properties**: Omega-3 fatty acids have anti-inflammatory effects in the body, including the brain. Chronic inflammation is associated with various brain disorders such as Alzheimer's disease and dementia. By reducing inflammation, omega-3 fatty acids may help protect the brain from damage.

3. **Neurotransmitter Function**: Omega-3 fatty acids are involved in the production and function of neurotransmitters, which are the chemical messengers that transmit signals in t

### Now, We Have a Fully Completed Holistic Health AI Assistant!